In [98]:
#utilities

# Show the figures / plots inside the notebook
# instead of io.imshow(gaussedImg) and plt.show() each time
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 


In [99]:
#initial step

from skimage import data, io, filters, feature, measure, transform
from skimage.color import rgb2gray
from matplotlib import pyplot as plt
import numpy as np

In [100]:
#preprocessing step

def preprocessingStep(img):

    grayImg=rgb2gray(img);

    if(np.max(grayImg) < 1):
        grayImg=grayImg*255

    gaussedImg=filters.gaussian(grayImg)

    addedConst=150
    imgH, imgW = np.shape(img)[:2]
    print(np.shape(img))
    randLvl = grayImg[int(imgH/100)][int(imgW/2)]
    thresh = randLvl + addedConst
    threImg=np.copy(gaussedImg)
    threImg[threImg>thresh]=255
    threImg[threImg<=thresh]=0
    
    return threImg

#     show_images([grayImg,gaussedImg,threImg])

In [101]:
#!(Edge detector) + find contours step

#calculate the distance between two points
def CalcDist(p1,p2):
    p1=np.copy(p1)
    p2=np.copy(p2)
    dist=np.sqrt((((p1-p2)**2).sum()))
    return dist

#calculate the perimeter of a contour
def CalcPerimeter(contour):
    ret=0
    prev=contour[-1]
    for cur in contour:
        ret+=CalcDist(cur,prev)
        prev=cur
    return ret

#caculate angle of three points using law of cosines
#arccos((P12^2 + P13^2 - P23^2) / (2 * P12 * P13))
def CalcAngle(points):
    p12=CalcDist(points[0],points[1])
    p13=CalcDist(points[2],points[1])
    p23=CalcDist(points[0],points[2])

    angle=np.arccos((p12**2+p13**2-p23**2)/(2*p12*p13))
    return angle*180/np.arccos(-1)

#caculate Area a polygon using Shoelace formula
def CalcArea(x,y):
    area=0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
    return area


maxAngle=160
minAngle=20

maxArea=3000000
minArea=100000

#check if the contour is likely to be a card
def CheckContour(contour):
    
    if(len(contour)!=5):
        return False
    
    area=CalcArea(contour[:4,1],contour[:4,0])
    if(area > maxArea or area < minArea):
        return False
    
    points=np.copy(contour)
    points=np.vstack([points, points[1]])    
    
    for i in range(4):
        angle=CalcAngle(points[i:i+3])
        if(angle >maxAngle or angle < minAngle):
            print(angle)
            return False;
    
    return True

def findCardsStep(threImg):
    
    contours=measure.find_contours(threImg)

    cards=[]
    for contour in contours:
        perimeter=CalcPerimeter(contour)
        approx=measure.approximate_polygon(contour,.08*perimeter)
        if(CheckContour(approx)==True):
            cards.append(approx)
            
    return cards

In [102]:
#sort the corners of the card
def SortCorners(approxContour):
    
    corners=np.copy(approxContour[:4])
    temp_rect=np.zeros((4,2))
    
    s = np.sum(corners, axis = -1)
    tl = np.argmin(s)
    br = np.argmax(s)

    dif = np.diff(corners, axis = -1)
    bl = np.argmin(dif)
    tr = np.argmax(dif)
    
    w=np.max(corners[:,1])-np.min(corners[:,1])
    h=np.max(corners[:,0])-np.min(corners[:,0])

    maxl=np.argmin(corners[:,1])
    maxr=np.argmax(corners[:,1])
    maxt=np.argmin(corners[:,0])
    maxb=np.argmax(corners[:,0])
    
    margin=.2
    print((1-margin)*h)
    if w <= ((1-margin)*h):
        print([w,h,tl,'vertical'])
        corners=np.roll(corners,-tl,axis=0)

    if w >= ((1+margin)*h):
        print('horizontal')
        print(bl)
        corners=np.roll(corners,-bl,axis=0)      
        
    if w > ((1-margin)*h) and w < ((1+margin)*h):
        print('diamond')
        print(tl)        
        if corners[maxr][0] <= corners[maxl][0]:
            corners=np.roll(corners,-maxt,axis=0)

        if corners[maxr][0] > corners[maxl][0]:
            corners=np.roll(corners,-maxl,axis=0)

    return corners    

In [103]:
row=350
col=250

def perspectiveStep(cards,threImg):

    for card in cards:
        dst=(SortCorners(card))[:,[1,0]].astype(int)
        src=np.array([
            [0,0],
            [col,0],
            [col,row],
            [0,row]
        ])
        transMatrix=transform.ProjectiveTransform()
        transMatrix.estimate(src,dst)
        warpedImg=transform.warp(threImg,transMatrix,output_shape=(row,col))
        warpedImg=warpedImg
        
        print(src)
        print(dst)
        print(card)
        
        return warpedImg

In [104]:
def applyAlgo(img):
    threImg=preprocessingStep(img)
    cards=findCardsStep(threImg)
    outImg=perspectiveStep(cards,threImg)
    out=[img,threImg]
    if outImg is not None:
        out.append(outImg)
    show_images(out)

In [105]:
#apply the algorithm on the dataset
import os

dire="../input/the-complete-playing-card-dataset/Images/Images/"
for filename in os.listdir(dire):
    print(dire+filename)
    img=io.imread(dire+filename);
    applyAlgo(img)